###Flujo completo de preprocesado y procesado para adecuar la imagen obtenida al modelo entrenado con imágenes. Se segmenta la señal binaria por latidos, se procesa cada latido y se pasa al modelo.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
a_tipo_F=("/content/drive/MyDrive/arritmias_final/Arritmia_tipo_F.jpeg")
a_tipo_Q=("/content/drive/MyDrive/arritmias_final/Arritmia_tipo_Q.jpeg")
a_tipo_S=("/content/drive/MyDrive/arritmias_final/Arritmia_tipo_S.jpeg")
a_tipo_V=("/content/drive/MyDrive/arritmias_final/Arritmia_tipo_V.jpeg")
a_tipo_N =("/content/drive/MyDrive/arritmias_final/Normal.jpeg")

arritmia_V = "/content/drive/MyDrive/arritmias.final/V.jpeg"
arritmia_S = "/content/drive/MyDrive/arritmias.final/S.jpeg"
arritmia_Q = "/content/drive/MyDrive/arritmias.final/Q.jpeg"
arritmia_F = "/content/drive/MyDrive/arritmias.final/F.jpeg"

In [3]:
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy import signal
import numpy as np
import cv2
from PIL import Image, ImageOps
import os
from tensorflow import keras
from scipy.signal import resample
import pandas as pd
import numpy as np
from PIL import Image
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import tensorflow as tf

In [4]:





# Funciones del primer script

#PREPROCESADO DE ECG

# Importar las bibliotecas necesarias
from PIL import Image
import cv2
import numpy as np
from scipy.signal import find_peaks
import os

# Función para convertir una imagen en escala de grises en una imagen binaria invertida
def img_to_binary(img_path):
    # Abre la imagen usando la biblioteca Pillow
    img = Image.open(img_path)
    # Convierte la imagen a escala de grises
    img_grayscale = img.convert('L')
    # Establece un umbral para convertir la imagen en una imagen binaria invertida
    threshold_value = 150
    img_binary = img_grayscale.point(lambda x: 0 if x < threshold_value else 255, '1')
    # Convierte la imagen binaria invertida en un array de numpy
    img_binary = np.array(img_binary).astype(np.uint8)
    # Invierte los valores de la imagen binaria
    img_inverted = Image.fromarray(255 - img_binary)

    return img_inverted

# Función para recortar la sección inferior de la imagen binaria invertida
def cut_signal(img_inverted):
    # Convierte la imagen binaria invertida en un array de numpy
    img_inverted = np.array(img_inverted)
    # Normaliza la imagen para mejorar la calidad de la señal
    img_norm = cv2.normalize(img_inverted, None, 0, 255, cv2.NORM_MINMAX)
    # Obtiene las dimensiones de la imagen
    height, width = img_inverted.shape[:2]
    # Define la región de interés (ROI) como la sección inferior de la imagen
    x = 0
    y = int(height * 0.715)
    w = width
    h = int(height * 0.2)
    seccion_inferior = img_norm[y:y+h, x:x+w]

    return seccion_inferior

# Función para convertir un trazo del ECG dilatado en picos del ECG y la distancia media entre ellos
def trazo_to_peaks(dilated_trazo):
    # Convierte el trazo del ECG dilatado en un array de numpy
    img_arr = np.asarray(dilated_trazo)
    # Calcula la señal promedio a lo largo del eje x del trazo del ECG
    signal = np.mean(img_arr, axis=0)
    # Normaliza la señal para mejorar la calidad de la señal
    signal = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
    # Establece un umbral para convertir la señal en una señal binaria
    threshold = 0.5
    binary_signal = np.where(signal > threshold, 1, 0)
    # Encuentra los picos de la señal binaria 
    peaks, _ = find_peaks(binary_signal, height=0.5)
    # Calcula la distancia media entre los picos
    mean_distance = int(np.mean(np.diff(peaks)))
    # Vuelve a encontrar los picos utilizando la distancia media como parámetro de distancia
    peaks, _ = find_peaks(binary_signal, height=0.5, distance=mean_distance)

    return peaks, mean_distance

# Función para extraer los latidos individuales del trazo del ECG
def signal_to_beats(dilated_trazo, peaks, mean_distance):
    # Calcula los intervalos RR a partir de los picos del ECG
    rr_intervals = np.diff(peaks)
    # Convierte el trazo del ECG dilatado en un array de numpy
    img_latidos = dilated_trazo
    latidos = []
    start = 0
    # Itera a través de los intervalos RR para extraer los latidos individuales del trazo del ECG
    for i in range(len(peaks) - 1):
        end = peaks[i+1] - int(mean_distance/2)
        latido = img_latidos[:, start:end]
        latidos.append(latido)
        start = end

    return latidos

# Función para guardar los latidos individuales como archivos de imagen
def save_latidos(latidos):
    # Define la ruta de la carpeta donde se guardarán los archivos de imagen de los latidos
    ruta_carpeta = os.path.join('/content', 'latidos')
    # Crea la carpeta si no existe
    os.makedirs(ruta_carpeta, exist_ok=True)
    # Itera a través de los latidos y guarda cada uno como un archivo de imagen
    for i, latido in enumerate(latidos):
        cv2.imwrite(f'latidos/latido_{i}.png', latido)

# Función que ejecuta todo el preprocesamiento del ECG
def executor(image_path):
    # Convierte la imagen en escala de grises en una imagen binaria invertida
    img_inverted = img_to_binary(image_path)
    # Recorta la sección inferior de la imagen binaria invertida
    seccion_inferior = cut_signal(img_inverted)
    # Convierte el trazo del ECG dilatado en picos del ECG y la distancia media entre ellos
    peaks, mean_distance = trazo_to_peaks(seccion_inferior)
    # Extrae los latidos individuales del trazo del ECG
    latidos = signal_to_beats(seccion_inferior, peaks, mean_distance) 
    # Guarda los latidos individuales como archivos de imagen
    save_latidos(latidos)




# Funciones del segundo script

#IMG_PREDICTIONS


# Función para realizar un preprocesamiento personalizado de la imagen de entrada
def custom_preprocessing(img_binary):
    # Convierte la imagen binaria en una imagen en pseudo-color (RGB)
    img_pseudocolor = np.stack((img_binary,)*3, axis=-1)
    # Preprocesa la imagen para que sea compatible con el modelo de clasificación
    img_preprocessed = preprocess_input(img_pseudocolor)
    return img_preprocessed

# Función para cargar el modelo de clasificación de arritmias
def charge_model(model_path):
    model_img_ecg = keras.models.load_model(model_path)
    return model_img_ecg

# Función para preprocesar la imagen de entrada para la clasificación de arritmias
def preprocess_image(image_path):
    # Carga la imagen en escala de grises
    img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Convierte la imagen en escala de grises a pseudo-color (RGB)
    img_pseudocolor = np.stack((img_gray,)*3, axis=-1)
    # Redimensiona la imagen al tamaño de entrada del modelo
    img_resized = cv2.resize(img_pseudocolor, (224, 224))
    # Normaliza la imagen
    img_normalized = img_resized / 255.0
    # Asegura que la imagen tenga la forma adecuada (1, 224, 224, 3)
    img_ready = np.expand_dims(img_normalized, axis=0)
    return img_ready

# Función para predecir la clase y probabilidad de arritmia en una imagen
def predict_arrhythmia(img_array, model):
    # Realiza una predicción utilizando el modelo de clasificación
    prediction = model.predict(img_array)
    # Obtiene la clase predicha y su probabilidad
    predicted_class = np.argmax(prediction, axis=1)
    predicted_probability = np.max(prediction, axis=1)
    return predicted_probability[0], predicted_class[0]

# Función para realizar la clasificación de arritmias en una imagen y devolver la clase y probabilidad predichas
def predictor_img(img, model):
    # Preprocesa la imagen
    img_array = preprocess_image(img)
    # Realiza la clasificación de arritmias en la imagen preprocesada
    predicted, predicted_probability = predict_arrhythmia(img_array, model)
    return predicted, predicted_probability

# Función que ejecuta todo el proceso de preprocesamiento y clasificación de arritmias en una imagen de ECG

def process_and_predict_ecg(image_path, model_path):
    # Realiza el preprocesamiento del ECG y guarda los latidos individuales como archivos de imagen
    executor(image_path)
    
   # Realiza la clasificación de arritmias en los latidos individuales utilizando el modelo de clasificación
    model = charge_model(model_path)
    folder_path = '/content/latidos'
    file_names = os.listdir(folder_path)
    
    for i, img in enumerate(file_names):
        predicted_probability, predictions = predictor_img(os.path.join(folder_path, img), model)
        
        if predictions == 2:
            print(f"Latido {i + 1}: No se detectó arritmia.")
        else:
            print(f"Latido {i + 1}: Se detectó arritmia de clase {predictions} con una probabilidad del {predicted_probability * 100:.2f}%.")




In [5]:


image_path = arritmia_V
model_path = '/content/drive/MyDrive/saved_models/model_ecg_escala_grises_mejorado.h5'
process_and_predict_ecg(image_path, model_path)


1/1 [==============================] - 1s 1s/step
Latido 1: No se detectó arritmia.
1/1 [==============================] - 1s 827ms/step
Latido 2: No se detectó arritmia.
1/1 [==============================] - 1s 570ms/step
Latido 3: No se detectó arritmia.
1/1 [==============================] - 1s 561ms/step
Latido 4: Se detectó arritmia de clase 1 con una probabilidad del 49.76%.
1/1 [==============================] - 1s 567ms/step
Latido 5: No se detectó arritmia.
1/1 [==============================] - 1s 568ms/step
Latido 6: No se detectó arritmia.
1/1 [==============================] - 1s 551ms/step
Latido 7: No se detectó arritmia.
1/1 [==============================] - 1s 594ms/step
Latido 8: No se detectó arritmia.
1/1 [==============================] - 1s 566ms/step
Latido 9: No se detectó arritmia.
1/1 [==============================] - 1s 588ms/step
Latido 10: No se detectó arritmia.
1/1 [==============================] - 1s 693ms/step
Latido 11: No se detectó arritmia.
1/1 

###Conclusiones:


Tras una exhaustiva investigación entendemos que el preprocesado le esta pasando un tipo de dato al modelo que no está siendo el adecuado. Procedemos a un nuevo análisis exploratorio en el colab de pruebas y exploración. Vemos mejores resultados con la parte numérica, pues conseguimos extraer correctamente la señal y convertirla en un data frame de N latidos por 187 columnas, con los valores de la amplitud de onda en cada momento.